# 爬取数据


In [2]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from openpyxl import Workbook, load_workbook

In [3]:
#循环构建榜单每一页的url
for i in range(11):
        print(f'正在爬取第{i}页,请稍等...')
        url = 'https://movie.douban.com/top250?start={}&filter='.format(i * 25)

正在爬取第0页,请稍等...
正在爬取第1页,请稍等...
正在爬取第2页,请稍等...
正在爬取第3页,请稍等...
正在爬取第4页,请稍等...
正在爬取第5页,请稍等...
正在爬取第6页,请稍等...
正在爬取第7页,请稍等...
正在爬取第8页,请稍等...
正在爬取第9页,请稍等...
正在爬取第10页,请稍等...


In [6]:
#获取每页25部电影链接
def getonepagelist(url, headers):
    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.raise_for_status()
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'html.parser')
        lsts = soup.find_all(attrs={'class': 'hd'})
        for lst in lsts:
            href = lst.a['href']
            time.sleep(0.5)
            getfilminfo(href, headers)
    except Exception as e:
        print('getonepagelist error:', e)

In [9]:
#获取每部电影具体信息
def getfilminfo(url, headers):
    filminfo = []
    r = requests.get(url, headers=headers, timeout=10)
    r.raise_for_status()
    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # 片名
    name = soup.find(attrs={'property': 'v:itemreviewed'}).text.split(' ')[0]
    # 上映年份
    year = soup.find(attrs={'class': 'year'}).text.replace('(','').replace(')','')
    # 评分
    score = soup.find(attrs={'property': 'v:average'}).text
    # 评价人数
    votes = soup.find(attrs={'property': 'v:votes'}).text
    
    infos = soup.find(attrs={'id': 'info'}).text.split('\n')
    # 提取所需信息
    director = scriptwriter = actor = filmtype = area = language = ''
    for info in infos:
        if '导演' in info:
            director = info.split(': ')[1]
        elif '编剧' in info:
            scriptwriter = info.split(': ')[1]
        elif '主演' in info:
            actor = info.split(': ')[1]
        elif '类型' in info:
            filmtype = info.split(': ')[1]
        elif '制片国家/地区' in info:
            area = info.split(': ')[1]
        elif '语言' in info:
            language = info.split(': ')[1]
    
    if '.' in area:
        area = area.split(' / ')[0]
        language = language.split(' / ')[0]
    else:
        area = area.split(' / ')[0]
        language = language.split(' / ')[0]

    if '大陆' in area or '香港' in area or '台湾' in area:
        area = '中国'
    if '戛纳' in area:
        area = '法国'

    # 时长
    times0 = soup.find(attrs={'property': 'v:runtime'}).text
    times = re.findall('\d+', times0)[0]
    
    filminfo.extend([name, year, score, votes, director, scriptwriter, actor, filmtype, area, language, times])
    filepath = 'TOP250.xlsx'
    insert2excel(filepath, filminfo)

In [15]:
#使用openpyxl将数据保存到Excel中
def insert2excel(filepath, allinfo):
    try:
        if not os.path.exists(filepath):
            tableTitle = ['片名', '上映年份', '评分', '评价人数', '导演', '编剧', '主演', '类型', '国家/地区', '语言', '时长(分钟)']
            wb = Workbook()
            ws = wb.active
            ws.title = 'sheet1'
            ws.append(tableTitle)
            wb.save(filepath)
            time.sleep(3)

        wb = load_workbook(filepath)
        ws = wb.active
        ws.title = 'sheet1'
        ws.append(allinfo)
        wb.save(filepath)
        return True
    except Exception as e:
        print("插入数据到Excel时出错:", e)
        return False


In [13]:
# 示例调用
filminfo = ['电影名', '2024', '9.0', '100000', '导演名', '编剧名', '主演名', '类型', '中国', '中文', '120']
filepath = 'TOP250.xlsx'
success = insert2excel(filepath, filminfo)
if success:
    print("数据成功插入到Excel文件中")
else:
    print("数据插入到Excel文件时出错")

数据成功插入到Excel文件中


# 爬虫


In [ ]:
#汇总运行，爬取数据
import os
import re
import time
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook, load_workbook
from fake_useragent import UserAgent
# os：用于操作系统相关的功能，如检查文件是否存在。
# re：用于正则表达式操作，提取文本中的特定模式。
# time：用于时间相关的功能，如休眠一段时间。
# requests：用于发送HTTP请求。
# BeautifulSoup：用于解析HTML和XML文档。
# openpyxl：用于操作Excel文件。
# fake_useragent：用于生成随机的User-Agent，以模拟不同的浏览器请求。

# 获取电影列表页数据
def getonepagelist(url, headers):
    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.raise_for_status()    #如果请求失败，抛出异常
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'html.parser')  #解析页面HTML。
        lsts = soup.find_all(attrs={'class': 'hd'}) #找到所有具有class属性值为'hd'的标签。
        hrefs = []         #用于存储电影详情页的链接。
        for lst in lsts:
            href = lst.a['href']
            hrefs.append(href)
            time.sleep(0.5)   #每获取一个链接后休眠0.5秒，防止请求过快。
        return hrefs       
    except Exception as e:
        print('getonepagelist error:', e)
        return []

# 获取每部电影的具体信息
def getfilminfo(url, headers):
    filminfo = []          # 用于存储电影信息的列表。
    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.raise_for_status()
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'html.parser')

        # 片名        .text：提取标签中的文本
        name = soup.find(attrs={'property': 'v:itemreviewed'}).text.split(' ')[0]
        # 上映年份
        year = soup.find(attrs={'class': 'year'}).text.replace('(','').replace(')','')
        # 评分
        score = soup.find(attrs={'property': 'v:average'}).text
        # 评价人数
        votes = soup.find(attrs={'property': 'v:votes'}).text
        
        infos = soup.find(attrs={'id': 'info'}).text.split('\n')
        # 提取所需信息
        director = scriptwriter = actor = filmtype = area = language = ''
        for info in infos:                            #初始化为空字符串，用于存储对应的信息。
            if '导演' in info:
                director = info.split(': ')[1]
            elif '编剧' in info:
                scriptwriter = info.split(': ')[1]
            elif '主演' in info:
                actor = info.split(': ')[1]
            elif '类型' in info:
                filmtype = info.split(': ')[1]
            elif '制片国家/地区' in info:
                area = info.split(': ')[1]
            elif '语言' in info:
                language = info.split(': ')[1]
        
        if '.' in area:        #如果国家/地区包含'.'，则进行特殊处理。
            area = area.split(' / ')[0]
            language = language.split(' / ')[0]
        else:
            area = area.split(' / ')[0]
            language = language.split(' / ')[0]

        if '大陆' in area or '香港' in area or '台湾' in area:
            area = '中国'
        if '戛纳' in area:
            area = '法国'

        # 时长
        times0 = soup.find(attrs={'property': 'v:runtime'}).text
        times = re.findall('\d+', times0)[0]
        
        filminfo.extend([name, year, score, votes, director, scriptwriter, actor, filmtype, area, language, times])
    except Exception as e:                         #将所有信息添加到filminfo列表中
        print('getfilminfo error:', e)
    
    return filminfo

# 将数据插入到Excel中
def insert2excel(filepath, allinfo):
    try:
        if not os.path.exists(filepath):
            tableTitle = ['片名', '上映年份', '评分', '评价人数', '导演', '编剧', '主演', '类型', '国家/地区', '语言', '时长(分钟)']
            wb = Workbook()         #创建一个新的Excel工作簿
            ws = wb.active
            ws.title = 'sheet1'
            ws.append(tableTitle)
            wb.save(filepath)
            time.sleep(3)

        wb = load_workbook(filepath)
        ws = wb.active
        ws.title = 'sheet1'
        for info in allinfo:
            ws.append(info)
        wb.save(filepath)
        return True
    except Exception as e:
        print("插入数据到Excel时出错:", e)
        return False

# 主程序
def main():
    base_url = 'https://movie.douban.com/top250'   #豆瓣电影Top 250页面的基本URL。
    headers = {'User-Agent': UserAgent().random}   #HTTP请求头，包含随机的User-Agent
    all_movie_info = []

    for start in range(0, 250, 25):  # 遍历所有页面，每页25个电影
        page_url = f'{base_url}?start={start}'
        movie_urls = getonepagelist(page_url, headers)
        for movie_url in movie_urls:
            movie_info = getfilminfo(movie_url, headers)
            if movie_info:
                all_movie_info.append(movie_info)
            time.sleep(1)  # 避免请求过快

    filepath = 'TOP250.xlsx'
    if insert2excel(filepath, all_movie_info):
        print("所有数据已成功插入到Excel文件中")
    else:
        print("插入数据到Excel文件时出错")

if __name__ == '__main__':
    main()



# 什么是User-Agent？
User-Agent 是一个 HTTP 头字段，包含了关于请求发起方（即用户使用的浏览器、操作系统和设备类型等信息）的详细描述。每次浏览器发送请求时，都会包含这个字段，以便服务器了解请求来自哪个设备、浏览器和操作系统。

## 可以使用fake_useragent库生成随机的User-Agent
from fake_useragent import UserAgent

#### 创建一个UserAgent对象
ua = UserAgent()

#### 获取一个随机的User-Agent字符串
random_user_agent = ua.random

#### 将随机的User-Agent添加到请求头
headers = {'User-Agent': random_user_agent}


In [22]:
# #了解哪些可以抓取
# import requests
# from urllib.robotparser import RobotFileParser

# def fetch_robots_txt(url):
#     try:
#         response = requests.get(url, timeout=10)
#         response.raise_for_status()
#         return response.text
#     except requests.RequestException as e:
#         print(f"Failed to fetch robots.txt: {e}")
#         return None

# def can_fetch(base_url, user_agent):
#     robots_url = base_url + "/robots.txt"
#     robots_txt = fetch_robots_txt(robots_url)
#     if robots_txt:
#         parser = RobotFileParser()
#         parser.parse(robots_txt.splitlines())
#         return parser.can_fetch(user_agent, base_url)
#     return False

# if __name__ == '__main__':
#     base_url = "https://movie.douban.com"
#     user_agent = "*"
#     check_url = base_url + "/top250"
#     if can_fetch(check_url, user_agent):
#         print(f"{user_agent} is allowed to fetch {check_url}")
#     else:
#         print(f"{user_agent} is not allowed to fetch {check_url}")


Failed to fetch robots.txt: 418 Client Error:  for url: https://movie.douban.com/top250/robots.txt
* is not allowed to fetch https://movie.douban.com/top250


In [2]:
# #可视化
# import pandas as pd
# from pyecharts import options as opts
# from pyecharts.charts import Bar

# data = pd.read_excel('TOP250.xlsx')

# # 提取年份数据
# year = data['上映年份'].value_counts().sort_index()